<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# SASRec 

Self-Attentive Sequential Recommendation (SASRec) [1], is a sequential recommendation system model that uses self-attention mechanisms to capture the sequential patterns in user-item interactions. It is designed to predict the next item a user is likely to interact with based on their previous interactions.

In [2]:
import sys
import torch

from recommenders.datasets import movielens 
from recommenders.models.unirec.model.sequential.sasrec import SASRec


print(f"System version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")

System version: 3.11.9 (main, Apr 19 2024, 16:48:06) [GCC 11.2.0]
PyTorch version: 2.3.1+cu121


In [3]:
df = movielens.load_pandas_df(size='100k', header=['userId', 'movieId', 'rating', 'timestamp'], genres_col="genre", local_cache_path=".")
df.head(5)

100%|██████████| 4.81k/4.81k [00:03<00:00, 1.50kKB/s]


,userId,movieId,rating,timestamp,genre
0,196,242,3.0,881250949,Comedy
1,186,302,3.0,891717742,Crime|Film-Noir|Mystery|Thriller
2,22,377,1.0,878887116,Children's|Comedy
3,244,51,2.0,880606923,Drama|Romance|War|Western
4,166,346,1.0,886397596,Crime|Drama


In [6]:
cate_df = df[["movieId", "genre"]].drop_duplicates()
print(cate_df.shape)
cate_df.head(5)

(1682, 2)


,movieId,genre
0,242,Comedy
1,302,Crime|Film-Noir|Mystery|Thriller
2,377,Children's|Comedy
3,51,Drama|Romance|War|Western
4,346,Crime|Drama


In [ ]:
# Extract all unique genres from the data
all_genres = set(genre for genre_string in cate_df['genre'] for genre in genre_string.split('|'))

# Create a mapping from genre to ID (1-based index)
genre_to_id = {genre: idx + 1 for idx, genre in enumerate(all_genres)}

# Map genres to IDs using the dynamic mapping
cate_df['cateId'] = cate_df['genre'].apply(lambda x: [genre_to_id[genre] for genre in x.split('|') if genre in genre_to_id])

print("Genre to ID Mapping:", {genre: id for genre, id in genre_to_id.items()})

# Display the result
cate_df.sort_values(by="movieId").head(5)


Genre to ID Mapping: {'Sci-Fi': 1, 'Western': 2, 'unknown': 3, 'Horror': 4, 'Film-Noir': 5, 'Thriller': 6, 'Mystery': 7, 'War': 8, 'Musical': 9, 'Romance': 10, 'Drama': 11, 'Animation': 12, "Children's": 13, 'Comedy': 14, 'Fantasy': 15, 'Action': 16, 'Crime': 17, 'Adventure': 18, 'Documentary': 19}


,movieId,genre,cateId
24,1,Animation|Children's|Comedy,"[12, 13, 14]"
172,2,Action|Adventure|Thriller,"[16, 18, 6]"
302,3,Thriller,[6]
48,4,Action|Comedy|Drama,"[16, 14, 11]"
78,5,Crime|Drama|Thriller,"[17, 11, 6]"


## Reference

\[1\] Wang-Cheng Kang, and Julian McAuley, *Self-Attentive Sequential Recommendation*, arXiv preprint arXiv:1808.09781, 2018. <br>

\[2\] Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Łukasz Kaiser, and Illia Polosukhin, *Attention is all you need*, in Advances in Neural Information Processing Systems, 5998–6008, 2017. <br>